In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import math
from datetime import datetime

In [6]:
loan_data_df = pd.read_csv("C:\\ADS\\LendingClubData\\loan_data\\loan_data.csv",low_memory=False)

In [13]:
loan_classification_df =  loan_data_df

In [14]:
loan_classification_df = loan_classification_df[loan_classification_df['fico_range_low'].notnull()]
loan_classification_df = loan_classification_df[loan_classification_df['fico_range_high'].notnull()]
loan_classification_df = loan_classification_df[loan_classification_df['fico_range_low'] > 660]
loan_classification_df['risk_score'] = (loan_classification_df['fico_range_low'] + loan_classification_df['fico_range_high']) / 2
loan_classification_df['term'] = loan_classification_df['term'].map({' 36 months': 36, ' 60 months' : 60})
loan_classification_df['int_rate'] = loan_classification_df['int_rate'].map(lambda x : x.replace('%', ''))
loan_classification_df['int_rate'] = loan_classification_df['int_rate'].map(float)
loan_classification_df = loan_classification_df[loan_classification_df['dti'].notnull() & loan_classification_df['loan_amnt'].notnull()]

In [32]:
loan_classification_df = loan_classification_df[loan_classification_df['emp_length'].notnull()]
loan_classification_df['emp_length'] = loan_classification_df['emp_length'].map(int)

In [33]:
loan_classification_df = loan_classification_df.dropna(axis=1, thresh=0.30 * loan_classification_df.shape[0])
loan_classification_df = loan_classification_df.dropna(axis=0, thresh = 0.30 * (loan_classification_df.shape[1]))

In [35]:
loan_classification_df = loan_classification_df[loan_classification_df['loan_amnt'].notnull() \
                                                              & loan_classification_df['purpose'].notnull() \
                                                             & loan_classification_df['risk_score'].notnull() \
                                                             & loan_classification_df['dti'].notnull() \
                                                             & loan_classification_df['addr_state'].notnull() \
                                                             & loan_classification_df['emp_length'].notnull()]

In [36]:
loan_classification_df = loan_classification_df[['loan_amnt', 'purpose', 'risk_score', 'dti', 'addr_state', 'emp_length']]
loan_classification_df['accepted'] = 1

In [40]:
rejected_data_df = pd.read_csv("C:\\ADS\\LendingClubData\\loan_data\\rejected_data.csv",low_memory=False,skiprows=1)

In [42]:
common_title = pd.Series(list(set(loan_classification_df['purpose']) & set(rejected_data_df['Loan Title'])))
rejected_data_df = rejected_data_df.loc[rejected_data_df['Loan Title'].isin(common_title)]
rejected_data_df = rejected_data_df[rejected_data_df['Policy Code'].notnull()]

In [43]:
rejected_data_df = rejected_data_df.rename(columns = {'Amount Requested' : 'loan_amnt', 'Loan Title' : 'purpose', 'Risk_Score' : 'risk_score', 'Debt-To-Income Ratio' : 'dti', 'Zip Code' : 'zip_code', 'State' : 'addr_state', 'Employment Length' : 'emp_length', 'Policy Code' : 'policy_code'})

In [45]:
rejected_data_df = rejected_data_df[rejected_data_df['emp_length'].notnull()]
rejected_data_df = rejected_data_df[rejected_data_df['emp_length'] != 'n/a']
rejected_data_df['emp_length'] = rejected_data_df['emp_length'].map({'4 years' : 4, '8 years' : 8, '10+ years' : 10, '< 1 year' : 0, '6 years' : 6, '2 years' : 2,
       '1 year' : 1, '3 years' : 3, '5 years' : 5, '7 years' : 7, '9 years' : 9})
rejected_data_df['emp_length'] = rejected_data_df['emp_length'].map(int)
rejected_data_df['dti'] = rejected_data_df['dti'].map(lambda x : x.replace('%', ''))

In [46]:
rejected_data_df = rejected_data_df[rejected_data_df['loan_amnt'].notnull() \
                                                              & rejected_data_df['purpose'].notnull() \
                                                             & rejected_data_df['risk_score'].notnull() \
                                                             & rejected_data_df['dti'].notnull() \
                                                             & rejected_data_df['addr_state'].notnull() \
                                                             & rejected_data_df['emp_length'].notnull()]

In [47]:
rejected_data_df = rejected_data_df[['loan_amnt', 'purpose', 'risk_score', 'dti', 'addr_state', 'emp_length']]
rejected_data_df['accepted'] = 0

In [53]:
data_list = [loan_classification_df, rejected_data_df]
classification_data = pd.concat(data_list, ignore_index=True)
classification_data = classification_data.dropna(axis=0)
purpose_dummies = pd.get_dummies(classification_data['purpose'])
addr_dummies = pd.get_dummies(classification_data['addr_state'])
classification_data = classification_data.join(purpose_dummies)
classification_data = classification_data.join(addr_dummies)
classification_data.drop('purpose', 1, inplace=True)
classification_data.drop('addr_state', 1, inplace=True)
classification_data.to_csv('C:\\ADS\\LendingClubData\\loan_data\\classification_data.csv', index = False)

In [73]:
loan_clustering_df =  loan_data_df

In [74]:
loan_clustering_df = loan_clustering_df[loan_clustering_df['fico_range_low'].notnull()]
loan_clustering_df = loan_clustering_df[loan_clustering_df['fico_range_high'].notnull()]
loan_clustering_df = loan_clustering_df[loan_clustering_df['fico_range_low'] > 660]
loan_clustering_df['risk_score'] = (loan_clustering_df['fico_range_low'] + loan_clustering_df['fico_range_high']) / 2
loan_clustering_df['term'] = loan_clustering_df['term'].map({' 36 months': 36, ' 60 months' : 60})
loan_clustering_df['int_rate'] = loan_clustering_df['int_rate'].map(lambda x : x.replace('%', ''))
loan_clustering_df['int_rate'] = loan_clustering_df['int_rate'].map(float)
loan_clustering_df = loan_clustering_df[loan_clustering_df['dti'].notnull() & loan_clustering_df['loan_amnt'].notnull()]

In [75]:
loan_clustering_df = loan_clustering_df[loan_clustering_df['emp_length'].notnull()]
loan_clustering_df = loan_clustering_df[loan_clustering_df['emp_length'] != 'n/a']
loan_clustering_df['emp_length'] = loan_clustering_df['emp_length'].map({'4 years' : 4, '8 years' : 8, '10+ years' : 10, '< 1 year' : 0, '6 years' : 6, '2 years' : 2,
       '1 year' : 1, '3 years' : 3, '5 years' : 5, '7 years' : 7, '9 years' : 9})
loan_clustering_df['emp_length'] = loan_clustering_df['emp_length'].map(int)

In [76]:
loan_clustering_df = loan_clustering_df.dropna(axis=1, thresh=0.30 * loan_clustering_df.shape[0])
loan_clustering_df = loan_clustering_df.dropna(axis=0, thresh = 0.30 * (loan_clustering_df.shape[1]))

In [77]:
loan_clustering_df = loan_clustering_df[['loan_amnt', 'purpose', 'risk_score', 'dti', 'grade', 'emp_length', 'annual_inc', 'term', 'sub_grade', 'int_rate']]

In [78]:
loan_clustering_df = loan_clustering_df[loan_clustering_df['int_rate'] <=25]
loan_clustering_df = loan_clustering_df[loan_clustering_df['loan_amnt'] <= 38000]

In [79]:
purpose_dummies = pd.get_dummies(loan_clustering_df['purpose'])
sub_grade_dummies = pd.get_dummies(loan_clustering_df['sub_grade'])
loan_clustering_df = loan_clustering_df.join(purpose_dummies)
loan_clustering_df = loan_clustering_df.join(sub_grade_dummies)
loan_clustering_df.drop('purpose', 1, inplace=True)
loan_clustering_df.drop('sub_grade', 1, inplace=True)

In [80]:
loan_clustering_df = loan_clustering_df[loan_clustering_df['annual_inc'].notnull()]

In [81]:
loan_clustering_df.to_csv('C:\\ADS\\LendingClubData\\loan_data\\clustering_data.csv', index = False)